# 🍫Tune your RAG data pipeline using parameter search and evaluate its performance

> ⚠️ This notebook can be run on your local machine or on a virtual machine and requires [Docker Compose](https://docs.docker.com/desktop/).
> Please note that it is unfortunately **not compatible with Google Colab** as the latter does not support Docker.

> 💡 This notebook allows you to perform parameter search, launch multiple runs and compare performance. Check out our [basic notebook](./evaluation.ipynb) if you want to configure a single run. 

In this notebook we demonstrate how to perform parameter search and automatically tune a Retrieval-Augmented Generation (RAG) system using [Fondant](https://fondant.ai).

We will:

1. Set up an environment and a [Weaviate](https://weaviate.io/platform) Vector Store
2. Define the sets of parameters that should be tried
3. Run the parameter search which automatically:
    * Runs an indexing pipeline for each combination of parameters to be tested
    * Runs an evaluation pipeline for each index
    * Collects results
5. Explore results

<div align="center">
<img src="../art/iteration.png" width="1000"/>
</div>

We will use [**Fondant**](https://fondant.ai), a hub and framework for easy and shareable data processing, as it has the following advantages for RAG evaluation:

- **Speed**
    - Leverage reusable RAG components from the [Fondant Hub](https://fondant.ai/en/latest/components/hub/) to quickly build RAG pipelines
    - [Pipeline caching](https://fondant.ai/en/latest/caching/) to speed up iteration on subsequent runs
    - Parallel processing out of the box to speed up processing of large datasets especially
    - Local development with the Docker Compose runner (used in this notebook)
- **Ease-of-use**
    - Easily adaptable: change parameters and swap [components](https://fondant.ai/en/latest/components/hub/) by changing only a few lines of code
    - Easily extendable: create your own [custom components](https://fondant.ai/en/latest/components/custom_component/) (eg. with different chunking strategies) and plug them into your pipeline
    - Reusable & shareable: reuse your processing components in different pipelines and share them with the [community](https://discord.gg/HnTdWhydGp)
- **Production-readiness**
    - Pipeline with dockerized steps ready to deploy to (managed) platforms such as _Vertex, SageMaker and Kubeflow_
    - Full data lineage and a [data explorer](https://fondant.ai/en/latest/data_explorer/) to check the evolution of data after each step
    - Ready to deploy to (managed) platforms such as _Vertex, SageMaker and Kubeflow_
 
Please share your experiences or let us know how we can improve through our [**Discord**](https://discord.gg/HnTdWhydGp) or on [**GitHub**](https://github.com/ml6team/fondant). And of course feel free to give us a [**star ⭐**](https://github.com/ml6team/fondant-usecase-RAG) if you like what we are doing!

## Set up environment

> ⚠️ This section checks the prerequisites of your environment. Read any errors or warnings carefully.

Ensure a Python between version 3.8 and 3.10 is available

In [ ]:
import sys
if sys.version_info < (3, 8, 0) or sys.version_info >= (3, 11, 0):
    raise Exception(f"A Python version between 3.8 and 3.10 is required. You are running {sys.version}")

Check if docker compose is installed and the docker daemon is running

In [ ]:
!docker compose version
!docker info && echo "Docker running"

### Set up Fondant

In [ ]:
!pip install -q -r ../requirements.txt --disable-pip-version-check && echo "Success"

In [1]:
from utils import get_host_ip, create_directory_if_not_exists, output_results, run_parameters_search

## Spin up the Weaviate vector store

> ⚠️ For Apple M1/M2 chip users:
> 
> - In Docker Desktop Dashboard `Settings -> Features in development`, make sure to uncheck `Use containerid` for pulling and storing images. More info [here](https://docs.docker.com/desktop/settings/mac/#beta-features)
> - Make sure that Docker uses linux/amd64 platform and not arm64 (cell below should take care of that)

In [ ]:
import os
os.environ["DOCKER_DEFAULT_PLATFORM"]="linux/amd64"

Run Weaviate with Docker compose

In [ ]:
!docker compose -f weaviate/docker-compose.yaml up --detach

Make sure you have Weaviate client v3

In [ ]:
!pip install -q "weaviate-client==3.*" --disable-pip-version-check && echo "Weaviate client installed successfully"

Make sure the vectorDB is running and accessible

In [ ]:
import logging
import weaviate

try:
    local_weaviate_client = weaviate.Client("http://localhost:8080")
    logging.info("Connected to Weaviate instance")
except weaviate.WeaviateStartUpError:
    logging.error("Cannot connect to weaviate instance, is it running?")

## Parameter search

Parameter search allows you try out different configurations of pipelines and compare their performance. Currently only grid search (which probes all possible combinations of different parameters) has been implemented but more will be added soon.

The first pipeline run is `pipeline_index.py`, which processes text data and loads it into the vector database. It consists of the following steps:

<div align="center">
<img src="../art/indexing_ltr.png" width="1000"/>
</div>

- [**HF Data Loading**](https://github.com/ml6team/fondant/tree/main/components/load_from_parquet): loads data from the Hugging Face Hub.
- [**Text Chunking**](https://github.com/ml6team/fondant/tree/main/components/chunk_text): divides the text into sections of a certain size and with a certain overlap
- [**Text Embedding**](https://github.com/ml6team/fondant/tree/main/components/embed_text): embeds each chunk as a vector.  
  💡 Can use different models / APIs. When using a HuggingFace model (the default)
- [**Write to Weaviate**](https://github.com/ml6team/fondant/tree/main/components/index_weaviate): writes data and embeddings to the vector store

The second pipeline is `pipeline_eval.py` which evaluates retrieval performance using the questions provided in your test dataset.

<div align=center>
<img src="../art/evaluation_ltr.png" width="1000"/>
</div>

- [**CSV Data Loading**](https://github.com/ml6team/fondant/tree/main/components/load_from_csv): loads the evaluation dataset (questions) from a csv file.
- [**Text Embedding**](https://github.com/ml6team/fondant/tree/main/components/embed_text): embeds each chunk as a vector.
  💡 Can use different models / APIs. When using a HuggingFace model (the default), use a machine with GPU for large datasets.
- [**Vector store Retrieval**](https://github.com/ml6team/fondant/tree/main/components/retrieve_from_weaviate): retrieves the most relevant chunks for each question from the vector store.
- [**Ragas evaluation**](https://github.com/ml6team/fondant/tree/0.8.0/components/evaluate_ragas): evaluates the retrieved chunks for each question with [RAGAS](https://docs.ragas.io/en/latest/index.html).
- [**Aggregate metrics**](https://github.com/ml6team/fondant-usecase-RAG/tree/main/src/components/aggregate_eval_results): Aggregate the results on a pipeline level.

> 💡 This notebook defaults to the first 1000 rows of the [wikitext](https://huggingface.co/datasets/wikitext) dataset for demonstration purposes, but you can load your own dataset using one the other load components available on the [**Fondant Hub**](https://fondant.ai/en/latest/components/hub/#component-hub) or by creating your own [**custom load component**](https://fondant.ai/en/latest/guides/implement_custom_components/). Keep in mind that changing the dataset implies that you also need to change the evaluation dataset used in the evaluation pipeline. 

### Set up parameter search

Select parameters to search over

- `chunk_sizes`: size of each text chunk, in number of characters ([chunk text](https://github.com/ml6team/fondant/tree/main/components/chunk_text) component)
- `chunk_overlaps`: overlap between chunks ([chunk text](https://github.com/ml6team/fondant/tree/main/components/chunk_text) component)
- `embed_models`: model used to embed ([embed text](https://github.com/ml6team/fondant/tree/main/components/embed_text) component)
- `top_ks`: number of retrieved chunks taken into account for evaluation ([retrieve](https://github.com/ml6team/fondant/tree/main/components/retrieve_from_weaviate) component)

In [ ]:
# parameter search
chunk_sizes = [256, 512]
chunk_overlaps = [10, 50]
embed_models = [("huggingface","all-MiniLM-L6-v2"), ("huggingface", "BAAI/bge-base-en-v1.5")]
top_ks = [2, 5]

⚠️ If you want to use ChatGPT you will need an [OpenAI API key](https://platform.openai.com/docs/quickstart) (see TODO below)

In [ ]:
# Define evaluation dataset to load (csv file with a "question" column)
extra_volumes = [str(os.path.join(os.path.abspath('.'), "evaluation_datasets")) + ":/data"]

# configurable parameters shared by indexing and evaluation pipeline (further below)
host_ip = get_host_ip() #get the host IP address to enable Docker access to Weaviate

BASE_PATH = "./data-dir"
BASE_PATH = create_directory_if_not_exists(BASE_PATH) #create a folder to store the pipeline data if it doesn't exist

fixed_args = {
    "base_path":BASE_PATH,
    "weaviate_url":f"http://{host_ip}:8080", # IP address 
}
fixed_index_args = {
    "n_rows_to_load":1000,
}
fixed_eval_args = {
    "csv_dataset_uri":"/data/wikitext_1000_q.csv", #make sure it is the same as mounted file
    "csv_separator":";",
    "evaluation_module": "langchain.llms",
    "evaluation_llm":"OpenAI",
    "evaluation_llm_kwargs":{"openai_api_key": ""}, #TODO Specify your key if you're using OpenAI
    "evaluation_metrics":["context_precision", "context_relevancy"]
}

### Run parameter search

> 💡 The first time you run a pipeline, you need to download a docker image for each component which may take a minute.

> 💡 Use a GPU to speed up the embedding step (when not using an external API)

> 💡 Steps that have been processed before are cached and will be skipped in subsequent runs.


In [ ]:
parameters_search_results = run_parameters_search(
    extra_volumes=extra_volumes,
    fixed_args=fixed_args,
    fixed_index_args=fixed_index_args,
    fixed_eval_args=fixed_eval_args,
    chunk_sizes=chunk_sizes,
    chunk_overlaps= chunk_overlaps,
    embed_models=embed_models,
    top_ks=top_ks,
)

## Explore Results

Compare the performance of your runs below

In [ ]:
results_df = output_results(results=parameters_search_results)
results_df

## Explore data

You can also check your data and results at each step in the pipelines using the Fondant data explorer. The first time you run the data explorer, you need to download the docker image which may take a minute. Afterwards you can access the data explorer at:

**http://localhost:8501/**

Enjoy the exploration! 🍫 

Press the ◼️ in the notebook toolbar to stop the explorer.

In [ ]:
from fondant.explore import run_explorer_app

run_explorer_app(base_path=fixed_args["pipeline_dir"])

## Clean up your environment

After your pipeline run successfully, you should clean up your environment and stop the weaviate database.

In [ ]:
!docker compose -f weaviate/docker-compose.yaml down

## Feedback

Please share your experience or let us know how we can improve through our 
* [**Discord**](https://discord.gg/HnTdWhydGp) 
* [**GitHub**](https://github.com/ml6team/fondant)

And of course feel free to give us a [**star** ⭐](https://github.com/ml6team/fondant) if you like what we are doing!